In [40]:
import sys
import pandas as pd
import urllib
from bs4 import BeautifulSoup as bs
import requests
import io
PROJECT_ROOT=r"C:\Users\Pedro\Desktop\Python Notes\Programs\Untitled Folder"
sys.path.insert(0, PROJECT_ROOT)

In [41]:
URL  =  "http://rotoguru1.com/cgi-bin/fyday.pl?week=WK&year=2017&game=dk&scsv=1"
weeks = list(range(1,18, 1))

In [42]:
df = pd.DataFrame()
for w in weeks:
    URL = URL.replace("WK", str(w))
    request = requests.get(URL)
    data = request.text
    temp_df=pd.read_csv(io.StringIO(bs(data, "html.parser").find("pre").text), sep=";")
    df = pd.concat([df, temp_df])
    URL = "http://rotoguru1.com/cgi-bin/fyday.pl?week=WK&year=2017&game=dk&scsv=1"
del temp_df
del weeks
del w
del df["GID"]

In [5]:
df.set_index(["Pos", "Name", "Year", "Week", "Oppt"], inplace=True)

In [6]:
defense = pd.DataFrame()
position = ["QB", "RB", "WR", "TE"]
for p in position:
    opt_rb = df.xs(p, level = 'Pos')
    opt_rb = opt_rb.groupby(["Week", "Oppt"]).sum()
    opt_rb = opt_rb.drop(columns=["DK salary"])
    opt_rb ["Pos"] = p
    defense = pd.concat([defense, opt_rb])
del p
del opt_rb
del position

In [7]:
opt_mean = round(defense.groupby(["Oppt", "Pos"]).mean(),2)
opt_mean = opt_mean.rename(columns={"DK points": "Oppt Average Points Allowed"})

In [8]:
df.reset_index(inplace=True)
opt_mean.reset_index(inplace=True)
df = pd.merge(df, opt_mean, on=["Pos", "Oppt"])
df.set_index(["Pos", "Name", "Year", "Week", "Oppt"], inplace=True)
del opt_mean

In [9]:
df_mean = df.drop(columns=["DK salary"])
df_mean = round(df_mean.groupby(["Name", "Pos", "Year"]).mean(), 2)
df_mean = df_mean.rename(columns={"DK points":"Avg points"})

In [10]:
df_std = df.drop(columns=["DK salary", "Oppt Average Points Allowed"])
df_std = round(df_std.groupby(["Name", "Pos", "Year"]).std(), 2)
df_std = df_std.rename(columns={"DK points":"STD"})

In [11]:
summary = pd.concat([df_mean, df_std], axis=1)
del df_std
del df_mean

In [12]:
cieling = summary["Avg points"] + summary["STD"]
floor = summary["Avg points"] - summary["STD"]

In [13]:
HL=pd.concat({"Cieling":cieling, "Floor": floor}, axis=1)
summary=pd.concat([summary, HL], axis=1)
del cieling
del floor

In [14]:
gp = df.groupby(["Name", "Pos", "Year"]).count()
gp = gp.drop(columns=["Team", "h/a", "DK points", "Oppt Average Points Allowed"])
gp = gp.rename(columns={"DK salary":"Games Played"})

In [15]:
summary=pd.concat([summary, gp], axis=1)
summary.sort_values(by=["Avg points"], ascending=False, inplace=True)
del gp

In [16]:
summary = summary[["Avg points", "STD", "Cieling", "Floor", "Games Played", "Oppt Average Points Allowed"]]
summary.sort_values(by=["Avg points"], ascending=False, inplace=True)

In [17]:
idx= pd.IndexSlice
temp = summary.loc[idx[:,"WR",:], idx["Oppt Average Points Allowed"]].transform(lambda x: x/2)
summary.update(temp, join="left", overwrite=True, filter_func=None, raise_conflict=False)
del temp

In [19]:
summary.query("Name == 'Forte, Matt'")

,,,Avg points,STD,Cieling,Floor,Games Played,Oppt Average Points Allowed
Name,Pos,Year,,,,,,
"Forte, Matt",RB,2017,10.2,7.3,17.5,2.9,12,24.9


In [433]:
#xs1 = summary.query("Pos != 'QB'")
#xs2 = summary.xs("QB", level="Pos")

In [434]:
#xs1.to_excel("Position_Draft_Card.xlsx")
#xs2.to_excel("QB_Draft_Card.xlsx")